In [2]:
### Investing and Rebalincing a Portfolio

# 1) Instantiate
# Build a portfolio with the following target allocations:
# - Amazon(AMZN) = 40%
# - Cisco(CSCO) = 30%
# - General Electric(GE) = 30%

targets = {
    'AMZN': 0.40, # Amazon
    'CSCO': 0.30, # Cisco
    'GE': 0.30 # General Electric
}

In [3]:
import pandas as pd
import numpy as np

# Manually created data frame
portfolio = pd.DataFrame(
    index=list(targets.keys()) + ['CASH'],
    data={
        'date': '2020-09-25',
        'price': [np.NaN, np.NaN, np.NaN, 1],
        'target': [0.4, 0.3, 0.3, 0],
        'allocation': [0, 0, 0, 1],
        'shares': [0, 0, 0, 10000],
        'market_value': [0, 0, 0, 10000]
    }
)
print(portfolio)

            date  price  target  allocation  shares  market_value
AMZN  2020-09-25    NaN     0.4           0       0             0
CSCO  2020-09-25    NaN     0.3           0       0             0
GE    2020-09-25    NaN     0.3           0       0             0
CASH  2020-09-25    1.0     0.0           1   10000         10000


In [4]:
# Portfolio Instantiation Function
def instantiate_portfolio(targets, starting_balance):

    # Initialize CASH to zero.  This is not a security and does not 
    # need to be rebalanced.  It is just an overflow buffer
    targets['CASH'] = 0
    # Convert input dict into a list
    tickers = list(targets.keys())

    df = pd.DataFrame(
        index=tickers,
        columns=[
            'date','price','target',
            'allocation','shares','market_value'
        ]
    )
    df.shares = 0
    df.market_value = 0
    df.allocation = 0
    df.update(
        pd.DataFrame
            .from_dict(targets, orient="index")
            .rename(columns={0: 'target'})
    )
    df.at['CASH','shares'] = starting_balance

    return df

# Function Test
targets = {
    'AMZN': 0.40, # Amazon
    'CSCO': 0.30, # Cisco
    'GE': 0.30 # General Electric
}

portfolio = instantiate_portfolio(targets, 10000)
print(portfolio)

     date price target  allocation  shares  market_value
AMZN  NaN   NaN    0.4           0       0             0
CSCO  NaN   NaN    0.3           0       0             0
GE    NaN   NaN    0.3           0       0             0
CASH  NaN   NaN      0           0   10000             0


In [5]:
# Function to update ticker prices
def update_prices(portfolio, prices):
    prices['CASH'] = 1
    portfolio.update(pd.DataFrame({'price': prices}))
    portfolio.date = prices.name
    portfolio.market_value = portfolio.shares * portfolio.price

# fake for right now
prices = pd.Series(
    name='2018-01-01',
    data={'AMZN': 945.21, 'CSCO': 30.52, 'GE': 29.27}
)

print(prices)
update_prices(portfolio, prices)

AMZN    945.21
CSCO     30.52
GE       29.27
Name: 2018-01-01, dtype: float64


In [6]:
print(portfolio)

            date   price target  allocation  shares market_value
AMZN  2018-01-01  945.21    0.4           0       0            0
CSCO  2018-01-01   30.52    0.3           0       0            0
GE    2018-01-01   29.27    0.3           0       0            0
CASH  2018-01-01       1      0           0   10000        10000


In [7]:
# Orders will calculate the buy and sell orders to achieve target balances
# Does not edit the original portfolio.
def get_order(portfolio):
    total_value = portfolio.market_value.sum()

    order = (
        (total_value * portfolio.target // portfolio.price)  #// is a floor divisor to return quotiant
        - portfolio.shares
    ).drop('CASH')

    return order

# Use function to get buy amounts
order = get_order(portfolio)
print(order)

AMZN      4
CSCO     98
GE      102
dtype: object


In [8]:
# Deposit

def deposit(portfolio, amount):
    portfolio.at['CASH', 'shares'] += amount
    portfolio.at['CASH', 'market_value'] = portfolio.at['CASH','shares']

deposit(portfolio, 1000)

order = get_order(portfolio)
print(order)

AMZN      4
CSCO    108
GE      112
dtype: object


In [9]:
# Simulate the execution of by/sell orders.

def simulate_process_order(portfolio,order):
    starting_cash = portfolio.at['CASH','shares']
    cash_adjustment = np.sum(order *portfolio.price)
    portfolio.shares += order
    portfolio.market_value = portfolio.shares * portfolio.price
    portfolio.at['CASH','shares'] = starting_cash - cash_adjustment
    portfolio.market_value = portfolio.shares * portfolio.price
    portfolio.allocation = (
        portfolio.market_value / portfolio.market_value.sum()
    )

simulate_process_order(portfolio, order)
print(portfolio)

            date   price target allocation  shares market_value
AMZN  2018-01-01  945.21    0.4   0.343713       4      3780.84
CSCO  2018-01-01   30.52    0.3   0.299651     108      3296.16
GE    2018-01-01   29.27    0.3   0.298022     112      3278.24
CASH  2018-01-01       1      0  0.0586145  644.76       644.76


In [10]:
import requests
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

API_KEY = os.environ.get('OPX_KEY')
TODAY = pd.Timestamp.today().normalize()

# GET_PRICE function
def get_price(ticker, outputsize="compact", most_recent=False):
    URL = 'https://www.alphavantage.co/query?'
    payload = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': ticker,
        'apikey': API_KEY,
        'outputsize': outputsize
    }
    r = requests.get(URL, params=payload)
    p = pd.DataFrame(r.json()['Time Series (Daily)']).T['4. close']
    df = pd.DataFrame({ticker: p.apply(float)})
    df.index = pd.to_datetime(df.index)
    if most_recent:
        return df.tail(1)
    return df

print(get_price('AMZN')[:10])

               AMZN
2020-09-28  3174.05
2020-09-25  3095.13
2020-09-24  3019.79
2020-09-23  2999.86
2020-09-22  3128.99
2020-09-21  2960.47
2020-09-18  2954.91
2020-09-17  3008.73
2020-09-16  3078.10
2020-09-15  3156.13


In [20]:
# Disecting the function above.
URL = 'https://www.alphavantage.co/query?'
payload = {
    'function': 'TIME_SERIES_DAILY_ADJUSTED',
    'symbol': 'AMZN',
    'apikey': API_KEY,
    'outputsize': 'compact'
}
r = requests.get(URL, params=payload)
print(r.json().keys())

dict_keys(['Meta Data', 'Time Series (Daily)'])


In [15]:
# This get_historical function will loop through each ticker and bind the 
# cleaned-up response to a main DataFrame object. The function will enable 
# us to fetch data and close prices for any number of listed companies 
# across any arbitrary length of time.
import datetime

def get_historical(tickers, start_date, end_date):
    df = pd.DataFrame(index=pd.date_range(start_date, end_date, freq="D"))
    for t in tickers:
        df = pd.concat([
            df,
            get_price(t, outputsize="full")],
            axis=1,
            join='outer'
        )
    df = df.fillna(method='ffill').dropna()
    return df

historical_prices = get_historical(
    tickers=['AMZN', 'CSCO', 'GE'],
    start_date=pd.Timestamp(2016, 1, 1),
    end_date=TODAY
)
print(historical_prices.tail())

               AMZN   CSCO    GE
2020-09-24  3019.79  37.85  6.06
2020-09-25  3095.13  38.45  6.11
2020-09-26  3095.13  38.45  6.11
2020-09-27  3095.13  38.45  6.11
2020-09-28  3174.05  39.13  6.20


In [16]:
prices = historical_prices.loc['2016-01-04']
print(prices)

AMZN    636.99
CSCO     26.41
GE       30.71
Name: 2016-01-04 00:00:00, dtype: float64


In [20]:
# Put the Portfolio into Action
portfolio = instantiate_portfolio(targets, 100000.00)
prices = historical_prices.loc['2017-01-01']
update_prices(portfolio, prices)
order = get_order(portfolio)
simulate_process_order(portfolio, order)
portfolio.market_value.sum()

print('Starting Portfolio\n\n',portfolio)


Starting Portfolio

            date   price target allocation  shares market_value
AMZN 2017-01-01  749.87    0.4   0.397431      53      39743.1
CSCO 2017-01-01   30.22    0.3   0.299782     992      29978.2
GE   2017-01-01    31.6    0.3   0.299884     949      29988.4
CASH 2017-01-01       1      0  0.0029025  290.25       290.25


In [21]:
# Test the Rebalance function across 2017 and execute orders on
# a quarterly-end frequency by using the Q offset alias from pandas

dates = pd.date_range('2017-01-01', '2017-12-31', freq="Q").tolist()
for d in dates:
    prices = historical_prices.loc[d]
    update_prices(portfolio, prices)
    order = get_order(portfolio)
    print(f'{d}:\n{order}')
    simulate_process_order(portfolio, order)

portfolio.market_value.sum()

2017-03-31 00:00:00:
AMZN     -4
CSCO    -24
GE      149
dtype: object
2017-06-30 00:00:00:
AMZN    -5
CSCO    63
GE      97
dtype: object
2017-09-30 00:00:00:
AMZN      0
CSCO    -83
GE      124
dtype: object
2017-12-31 00:00:00:
AMZN     -7
CSCO    -79
GE      589
dtype: object


111030.14

In [22]:
print(portfolio)

           date    price target allocation   shares market_value
AMZN 2017-12-31  1169.47    0.4   0.389718       37      43270.4
CSCO 2017-12-31     38.3    0.3   0.299763      869      33282.7
GE   2017-12-31    17.45    0.3    0.29987     1908      33294.6
CASH 2017-12-31        1      0  0.0106498  1182.45      1182.45
